In [ ]:
#!/usr/bin/env python3
"""experiment_viterbi.ipynb
James Gardner 2020
ANU / Melbourne Uni

various assorted pieces of analysis of data from photodiode
reading the changing pattern of the optical microphone interferometer
"""

import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm_notebook as tqdm
import scipy.io.wavfile as wavfile

In [ ]:
def fourier_spectrum_2(signal, fps, return_spectrum=False, cutter=2, remove_mains=False,
                     produce_plot=False, out_plot_name='tmp.pdf', out_plot_title=''):
    """finds fourier spectrum of signal time series as numpy array,
    has functionality to return and/or plot the sectrum (both default off),
    built from code originally found in tracker_time_series.ipynb
    
    note: based off of fourier_spectrum, careful with older scripts
    """   
    signal_frames = len(signal)
    
    # will drop two frames later, fps: frames per second
    total_time = (signal_frames-cutter)/fps
    t = np.linspace(0,total_time,signal_frames)
    dt = t[1] - t[0]

    yf = np.fft.fft(signal)
    # normalised-absolute value of FT'd signal
    nrm_abs_yf = 2/signal_frames*np.abs(yf)
    # values at the centre of each frequency bin
    freq_scale = np.fft.fftfreq(len(yf),dt)
    # real signals are symmetric about 0 in frequency domain
    freq_scale_positive = freq_scale[cutter:signal_frames//2]
    # frequency distribution values on positive side
    freq_prob = nrm_abs_yf[cutter:signal_frames//2]
    
    # freqscale 0, +inf, -inf, -0
    #print(freq_scale[:10], freq_prob[:10])#,freq_scale[-5])
    
    if produce_plot:
        
        # tune out 50Hz mains noise
        # closest value in scale is index 5
        #to_go = None#(5,)#tuple(range(10))#(5,)
        # mains_mark
        to_go = ()
        
        if remove_mains:
            mm_0, mm_1 = None, None
            for i, f in enumerate(freq_scale_positive):
                if f > 50:
                    mm_0, mm_1 = i-1, i
                    break
            mm_f_0 = freq_scale_positive[mm_0]
            mm_f_1 = freq_scale_positive[mm_1]
            if abs(mm_f_0-50) < abs(mm_f_1-50):
                to_go = (mm_0,)
            else:
                to_go = (mm_1,)
            
        #print(to_go)
        to_go = tuple(to_go[0]+i for i in (-1,0,1))
        
        freq_prob_mained = np.delete(freq_prob, to_go)
        freq_scale_positive_mained = np.delete(freq_scale_positive, to_go)        
        
        #print(np.abs(yf)[:7])
        to_go_pandn = to_go + tuple(len(yf)-i for i in to_go)
        #print(to_go_pandn)
        
        yf_mained = np.delete(yf, to_go_pandn)
        t_mained = np.delete(t, to_go_pandn)
        iyfm = np.fft.ifft(yf_mained)
        
        fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(21,14))
        ax0,ax1,ax2,ax3 = axes[0,0],axes[1,0],axes[0,1],axes[1,1]
        ax0.plot(t, signal)
        ax0.set(title='signal: {}'.format(out_plot_title),ylabel='signal strength',xlabel='time, t')
        ax2.plot(t_mained,iyfm)
        ax2.set(title='i mained signal: {}'.format(out_plot_title),ylabel='signal strength',xlabel='time, t')
        # signal average value gives magnitude of frequency = 0 term
        # simple fix is to drop first two bins, otherwise need to shift signal
        ax1.plot(freq_scale_positive,freq_prob)
        ax1.set(title='discrete FFT',ylabel='freq strength in signal',xlabel='frequency, f')        
        ax3.plot(freq_scale_positive_mained,freq_prob_mained)
        ax3.set(title='mained discrete FFT',ylabel='freq strength in signal',xlabel='frequency, f')
        plt.savefig(out_plot_name,bbox_inches='tight')
        plt.clf()
    
    if return_spectrum:
        return freq_prob, freq_scale_positive  

In [ ]:
def cut_mains(signal, fps, cutter=2):
    """returns signal with the 50Hz mains cut out
    
    NB: this is a bad signal processing technique and should not be used
    """
    signal_frames = len(signal)
    
    total_time = (signal_frames-cutter)/fps
    t = np.linspace(0,total_time,signal_frames)
    dt = t[1] - t[0]

    yf = np.fft.fft(signal)
    # normalised-absolute value of FT'd signal
    nrm_abs_yf = 2/signal_frames*np.abs(yf)
    # values at the centre of each frequency bin
    freq_scale = np.fft.fftfreq(len(yf),dt)
    # real signals are symmetric about 0 in frequency domain
    freq_scale_positive = freq_scale[cutter:signal_frames//2]
    # frequency distribution values on positive side
    freq_prob = nrm_abs_yf[cutter:signal_frames//2]

    # tune out 50Hz mains noise
    to_go = ()
    mm_0, mm_1 = None, None
    for i, f in enumerate(freq_scale_positive):
        if f > 50:
            mm_0, mm_1 = i-1, i
            break
    mm_f_0 = freq_scale_positive[mm_0]
    mm_f_1 = freq_scale_positive[mm_1]
    if abs(mm_f_0-50) < abs(mm_f_1-50):
        to_go = (mm_0,)
    else:
        to_go = (mm_1,)

    #print(to_go)
    #to_go = tuple(to_go[0]+i for i in (-1,0,1))

    freq_prob_mained = np.delete(freq_prob, to_go)
    freq_scale_positive_mained = np.delete(freq_scale_positive, to_go)        

    #print(np.abs(yf)[:7])
    to_go_pandn = to_go + tuple(len(yf)-i for i in to_go)
    #print(to_go_pandn)

    yf_mained = np.delete(yf, to_go_pandn)
    t_mained = np.delete(t, to_go_pandn)
    iyfm = np.fft.ifft(yf_mained)

    return np.abs(iyfm), t_mained

In [ ]:
def viterbi_pathfinder(grid, scanning_range=3):
    """find the highest scoring path through the grid, left-to-right,
    as by the viterbi algorithm, with connections plus-minus the scanning_range;
    returns score grid for best path to each node and a bitmap of the total best path
    
    NB: this is ripped from previous scripts
    """
    # normalised grid, algorithm goal is to maximise product of values
    ngrid  = grid/np.max(grid)
    # logarithm avoids underflow, equvivalent to maximise sum of log of values
    lngrid = np.log(ngrid)
    
    long_timesteps = grid.shape[1]

    # keep track of running scores for best path to each node
    score_grid  = np.copy(lngrid)
    pathfinder_flag = len(lngrid[:,0])
    # pathfinder stores the survivor paths, i.e. the previous best step
    # to allow back-tracking to recover the best total path at the end
    pathfinder = np.full(np.shape(lngrid), pathfinder_flag)
    # pathfinder flag+1 for reaching the first, 0-index column        
    pathfinder[:,0] = pathfinder_flag+1       

    # implementation of the viterbi algorithm itself
    # finding the best path to each node, through time
    # see: https://www.youtube.com/watch?v=6JVqutwtzmo
    for j in range(1,long_timesteps):
        for i in range(len(score_grid[:,j])):
            # index values for where to look relative to i in previous column
            k_a = max(0, i-scanning_range) 
            k_b = min(len(score_grid[:,j-1])-1,
                      i+scanning_range)
            window = score_grid[:,j-1][k_a:k_b+1]
            # find the best thing nearby in the previous column ...
            window_score = np.max(window)
            window_ref   = k_a+np.argmax(window)
            # ... and take note of it, summing the log of values
            score_grid[i][j] += window_score
            pathfinder[i][j] = window_ref 

    # look at the very last column, and find the best total ending ...
    best_score  = np.max(score_grid[:,-1])
    best_end = np.argmax(score_grid[:,-1])
    # ... and retrace its steps through the grid
    best_path_back = np.full(long_timesteps,pathfinder_flag+2)
    best_path_back[-1] = best_end
    # best_path_back is the viterbi path, the highest scoring overall 
    # path_grid is the binary image of the viterbi path taken
    path_grid = np.zeros(np.shape(ngrid))
    tmp_path = pathfinder[best_end][-1]

    for j in reversed(range(0,long_timesteps-1)):
        path_grid[tmp_path][j] = 1
        # take pathfinder value in current step and follow it backwards
        best_path_back[j] = tmp_path    
        tmp_path = pathfinder[tmp_path][j]

    # make sure we got all the way home
    # (that the retrace found the initial edge)
    assert tmp_path == pathfinder_flag+1
    
    return score_grid, path_grid

In [ ]:
def absmax(axis):
    """returns the maximum absolute value along an np.axis"""
    return max(abs(np.min(axis)), abs(np.max(axis)))

In [ ]:
# create injected meandering signal to test viteri analysis

# webcam options
# meander_amp, meander_decay, meander_freq = 9, 0.01, 0.005
# filetag = 'webcam'

# podo options
meander_amp, meander_decay, meander_freq = 300, 0.01, 0.01
filetag = 'podo'

# 5 minutes duration
duration = 300
fps = 16000
long_timesteps = fps*duration

bin_time = np.linspace(0,300,long_timesteps)
# meander is the long scale change in the sine frequency
meander = lambda x: meander_amp*(
    np.exp(-x*meander_decay)*np.sin(meander_freq*2*np.pi*x))

initial_frequency = 5
# initial_frequency = 440
wandering_freqs = initial_frequency + meander(bin_time)

plt.figure(figsize=(14, 7))
plt.plot(bin_time, wandering_freqs)
# plt.xlim(0, 60)
plt.title('{} viterbi test: injected frequency versus time'.format(filetag))
plt.xlabel('time, t / s')
plt.ylabel('injected signal frequency, f / Hz')
plt.savefig('wandering_{}.pdf'.format(filetag))
plt.show()

big_n = bin_time.shape[0]
together = np.zeros((100, 2))
together[:, 0], together[:, 1] = (bin_time[::big_n//99],
                                  wandering_freqs[::big_n//99])
# careful, wandering_webcam is the frequency, not the actual signal
np.savetxt('wandering_{}.csv'.format(filetag), together, delimiter=',')

# sin(2*pi*f*t)
# therefore sin(g(t)), 2*pi*f = dg/dt
# sin(2*pi*\int{f(t)dt}) gives f = f(t)
# and cumsum is discrete \int (integral)
# https://au.mathworks.com/matlabcentral/answers/217746-
# implementing-a-sine-wave-with-linearly-changing-frequency

inj_signal = np.sin(2*np.pi*np.cumsum(wandering_freqs)/fps)
# this is the signal to play (i.e. inject) into the interferometer
wavfile.write('inj_{}.wav'.format(filetag), int(fps), inj_signal)

In [ ]:
# read in photodiode (aka podo) data as time series
filename = 'podo_viterbi_test.csv'
filetag = 'viterbi_test'

mega = np.genfromtxt(filename,delimiter=',')
times = mega[:,0]
long_signal = mega[:,1]
fps = len(mega)/mega[-1,0]

total_frames = len(long_signal)
duration = total_frames/fps
print('duration: {:.1f}s, fps: {:.1f}/s, frames: {}'.format(duration, fps, total_frames))

In [ ]:
# save an initial audio recording of the response
wavfile.write('podo_{}_raw.wav'.format(filetag), int(fps),
              long_signal.astype('float32')/absmax(long_signal))

In [ ]:
# processing a window of the time series
window_start = 5
window_duration = 0.5
window_start_frame = int(window_start/duration*total_frames)
window_size = int(window_duration/duration*total_frames)

window_signal = long_signal[window_start_frame:window_start_frame+window_size]
window_times = times[window_start_frame:window_start_frame+window_size]
window_prob, window_fscale = fourier_spectrum_2(window_signal, fps, return_spectrum=True)

fig, (ax0,ax1) = plt.subplots(2,figsize=(14,7))
ax0.plot(window_times, window_signal)
ax0.set(title='window signal',ylabel='photodiode reading',xlabel='time, t / s')
ax1.plot(window_fscale, window_prob)
ax1.set(ylabel='fourier strength',xlabel='frequency, f / Hz')
plt.savefig('podo_{}_window.pdf'.format(filetag))
plt.clf()

In [ ]:
# apply viterbi analysis to long_signal

long_timesteps = 600
scanning_range = 3

# q,r = divmod(a,b) s.t. a = q*b+r
bin_frames, bin_remainder = divmod(total_frames,long_timesteps)
# bin_duration = bin_frames/fps
# acts as flag to stop short of the remainder, which is lost
bin_last = total_frames - bin_remainder
# always has long_timesteps number of chunks
bin_signals = [long_signal[i: i+bin_frames] for i in range(0, bin_last, bin_frames)]

# creating the signal grid
grid_frames = bin_frames//2
grid = np.zeros((grid_frames,long_timesteps))

for i, signal in enumerate(tqdm(bin_signals)):
    # columns are each spectrum, rows are frequency through time
    col, freq_scale_cut = fourier_spectrum_2(signal, fps, return_spectrum=True,cutter=0)
    grid[:,i] = col
    
# fgrid ignores the first 100 bins, to avoid mains noise
fgrid = grid[100:,:]
ngrid = fgrid/np.max(fgrid)    

path_grid = viterbi_pathfinder(fgrid, scanning_range)[1]
print('viterbi path found')

In [ ]:
# save plots of grid and viterbi path through it

plt.figure(figsize=(7,14))
plt.imshow(ngrid, cmap='viridis');
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')  
# plt.xticks(np.arange(long_timesteps),xtick_labels_1, rotation=90)
# plt.yticks(np.arange(grid_frames),ytick_labels_1)
cbar_fraction = 0.025
cbar = plt.colorbar(fraction=cbar_fraction) 
cbar.set_label('normalised frequency distribution')
plt.title('{}\n fourier spectrum of signal binned over time\n'.format(filename))
plt.ylabel('signal frequency, f / Hz')
plt.xlabel('long time duration, t / s')
plt.savefig('expt_ngrid_{}.pdf'.format(filetag),bbox_inches='tight')
plt.clf()

plt.figure(figsize=(7,14))
plt.imshow(path_grid, cmap='viridis');
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')  
# plt.xticks(np.arange(long_timesteps),xtick_labels_1, rotation=90)
# plt.yticks(np.arange(grid_frames),ytick_labels_1)
plt.title('{}\n viterbi path through signal grid'.format(filename))
plt.ylabel('signal frequency, f / Hz')
plt.xlabel('long time duration, t / s')
plt.savefig('expt_viterbi_path_{}.pdf'.format(filetag),bbox_inches='tight')
plt.clf()

In [ ]:
# find power spectral density (aka psd) of noise
# requires noise recording, like 14_6 to be present
filename = 'podo_14_6.csv'

mega = np.genfromtxt(filename,delimiter=',')
times = mega[:,0]
long_signal = mega[:,1]
fps = len(mega)/mega[-1,0]

c0, c1 = 10000, 400000
cut = long_signal[c0:c1]
cut_time = times[c0:c1]-times[c0]

fig, (ax0,ax1) = plt.subplots(2,figsize=(14,7))
ax0.plot(cut_time, cut)
ax0.set(title='example of noise from no sound, laser on',ylabel='photodiode reading',xlabel='time, t / s')
ax1.psd(cut, int(2**13), fps)
plt.savefig('podo_noise_psd.pdf')
ax1.set_xlim(0, 2000)
plt.savefig('podo_noise_psd_zoom.pdf')
plt.clf()

In [ ]:
# sanity checks for basic wav reading and writing

rate, signal = wavfile.read('source_feynman(1).wav')
# mono np.nonzero(signal[:,0] - signal[:,1])
signal = signal[:,0]

mono_signal = signal[:,0]
wavfile.write('feynamn_nochanges.wav', rate, mono_signal)

amplitude = np.iinfo(np.int16).max
squeeze = signal/absmax(signal)*amplitude/2
# scaled_data = amplitude * (squeeze-squeeze.mean())
# shift = signal+signal.mean()/1000

wavfile.write('feynman_direct.wav', rate, squeeze)

# decrease rate from 44100 to around 16000
audio_expt_rate = int(fps)
# rate/audio_expt_rate ~= 2.7 
rate_ratio = rate/audio_expt_rate
# sample every rate_ratio values
those = np.round(rate_ratio*np.arange(len(signal))).astype(int)
those = those[those < len(signal)]
sampled_signal = signal[those]

wavfile.write('feynman_sampled.wav', audio_expt_rate, sampled_signal)

In [ ]:
# confirming that slower fps and digitisation
# doesn't significantly impact audio signal

infile_name = 'source_a440.wav'
outfile_name = None#'a440_digi.wav'
rate, signal = wavfile.read(infile_name)
# mono np.nonzero(signal[:,0] - signal[:,1])
if len(signal.shape) > 1:
    signal = signal[:,0]

# decrease rate from 44100 to around 16000
audio_expt_rate = int(fps)
if rate > audio_expt_rate + 1000:
    # rate/audio_expt_rate ~= 2.7 
    rate_ratio = rate/audio_expt_rate
    print('rate_ratio:',rate_ratio)
    # sample every rate_ratio values
    those = np.round(rate_ratio*np.arange(len(signal))).astype(int)
    those = those[those < len(signal)]
    sampled_signal = signal[those]
else:
    sampled_signal = signal
    audio_expt_rate = rate

# sampled_signal_n = sampled_signal/absmax(sampled_signal)
# sampled_signal_n -= sampled_signal_n.mean()

# bin signal into channels
intensity_channels = 100
# centre = sampled_signal.mean()
m0, m1 = sampled_signal.min(), sampled_signal.max()
bins = np.linspace(m0-1e-10, m1+1e-10, intensity_channels)
bin_lookup = np.digitize(sampled_signal, bins)
digi_signal = bins[bin_lookup]/m1
# digi_signal = digi_signal/intensity_channels
# digi_signal = (digi_signal-0.5)*2*absmax(sampled_signal)+centre

# data_to_audio = digi_signal

# squeeze = data_to_audio/data_to_audio.max()
# amplitude = np.iinfo(np.int16).max
# scaled_data = amplitude * (squeeze-squeeze.mean())

if outfile_name is None:
    outfile_name = '{}_digi.wav'.format(infile_name[:-4])
wavfile.write(outfile_name, audio_expt_rate, digi_signal)

fig, (ax0, ax1) = plt.subplots(2,figsize=(14,14))
# a,b,b1 = 0, int(len(signal)/100), int(len(signal)/40+10000)
a, b = 0, int(len(signal)/50)
ax0.plot(signal[a:b])
ax1.plot(digi_signal[a:b])
plt.savefig('a440_digitised_simulation.pdf')
plt.show()
plt.clf()

In [ ]:
# def fourier_spectrum(signal, fps, return_spectrum=False,
#                      produce_plot=False, out_plot_name='tmp.pdf', out_plot_title=''):
#     """finds fourier spectrum of signal time series as numpy array,
#     has functionality to return and/or plot the sectrum (both default off),
#     built from code originally found in tracker_time_series.ipynb
    
#     note: please use fourier_spectrum_2, this is here for compatibility
#     """   
#     signal_frames = len(signal)
#     # will drop two frames later, fps: frames per second
#     total_time = (signal_frames-2)/fps
#     t = np.linspace(0,total_time,signal_frames)
#     dt = t[1] - t[0]

#     yf = np.fft.fft(signal)
#     # normalised-absolute value of FT'd signal
#     nrm_abs_yf = 2/signal_frames*np.abs(yf)
#     # values at the centre of each frequency bin
#     freq_scale = np.fft.fftfreq(len(yf),dt)
#     # real signals are symmetric about 0 in frequency domain
#     freq_scale_positive = freq_scale[:signal_frames//2]
#     # frequency distribution values on positive side
#     freq_prob = nrm_abs_yf[:signal_frames//2]
    
#     if produce_plot:
#         fig, (ax0,ax1) = plt.subplots(2,figsize=(14,14))
#         ax0.plot(t,signal)
#         ax0.set(title='signal: {}'.format(out_plot_title),ylabel='signal strength',xlabel='time, t')
#         # signal average value gives magnitude of frequency = 0 term
#         # simple fix is to drop first two bins, otherwise need to shift signal
#         ax1.plot(freq_scale_positive[2:],freq_prob[2:])
#         ax1.set(title='discrete FFT',ylabel='freq strength in signal',xlabel='frequency, f')
#         plt.savefig(out_plot_name,bbox_inches='tight')
#         plt.clf()
    
#     if return_spectrum:
#         return freq_prob[2:], freq_scale_positive[2:]        

In [ ]:
# # defunct, poor treatment of window'ed signal

# window_frames = len(window_signal)
# cutter = 0
# t = window_times
# dt = t[1] - t[0]

# yf = np.fft.fft(window_signal)
# nrm_abs_yf = 2/window_frames*np.abs(yf)
# freq_scale = np.fft.fftfreq(len(yf),dt)
# # freqscale: 0, +inf, -inf, -0
# # freq_scale_positive = freq_scale[cutter:window_frames//2]
# # freq_prob = nrm_abs_yf[cutter:window_frames//2]

# # window_prob, window_fscale = yf, freq_scale

# # how far above 100Hz to cut the signal
# low_pass_cut = 4
# freq_noise_threshold = 0.5

# # treated_fscale = low_pass_fscale[low_pass_prob > freq_noise_threshold]
# treated_prob = np.copy(yf)
# treated_prob[nrm_abs_yf < freq_noise_threshold] = 0
# treated_prob[(-100 - low_pass_cut < freq_scale) &
#              (freq_scale < 100 + low_pass_cut)] = 0

# treated_signal = np.abs(np.fft.ifft(treated_prob))

# wavfile.write('podo_{}_raw.wav'.format(filetag), int(fps), long_signal/200)
# wavfile.write('podo_{}_treated.wav'.format(filetag), int(fps), treated_signal)

In [ ]:
# # defunct: more notched cuts of mains

# mained_signal, mained_times = cut_mains(window_signal, fps)
# # prob, scale = fourier_spectrum_2(cut, fps, produce_plot=True, remove_mains=True,
# #                                  out_plot_name='cut2.pdf', return_spectrum=True, cutter=4000)

# fig, (ax0, ax1) = plt.subplots(2,figsize=(14,14))
# ax0.plot(cut)
# ax1.plot(mained_signal)
# # plt.savefig('a440_mained_experiment.pdf')
# # plt.clf()
# plt.show()


# prob_initial_max = window_fscale[np.argmax(window_prob)]

# if abs(prob_initial_max - 50) < 1:
#     window_fscale_1  = np.delete(window_fscale, np.argmax(window_prob))
#     window_prob_1 = np.delete(window_prob, np.argmax(window_prob))
#     print('removed mains: {:.4f}'.format(prob_initial_max))
    
#     prob_initial_max_1 = window_fscale_1[np.argmax(window_prob_1)]

#     if abs(prob_initial_max_1 - 100) < 1:
#         window_fscale_2  = np.delete(window_fscale_1, np.argmax(window_prob_1))
#         window_prob_2 = np.delete(window_prob_1, np.argmax(window_prob_1))
#         print('removed mains 1st harmonic: {:.4f}'.format(prob_initial_max_1))
#     else:
#         window_fscale_2, window_prob_2 = window_fscale_1, window_prob_1
#         print(prob_initial_max_1,'not mains 1st harmonic?')    
    
# else:
#     window_fscale_2, window_prob_2 = window_fscale, window_prob
#     print(prob_initial_max,'not mains?')
    
# print(window_fscale_2[np.argmax(window_prob_2)])